In [1]:
if 'libraries_imported' not in globals():
    libraries_imported = False

if libraries_imported == False:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os
    from datetime import datetime, timedelta
    from datetime import datetime
    from scipy.interpolate import make_interp_spline
    import plotly.graph_objects as go
    from scipy.signal import butter, filtfilt
    import pandas as pd
    from google.auth import default
    import seaborn as sns
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from keras_tuner import RandomSearch, HyperModel
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.callbacks import EarlyStopping
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from keras_tuner import RandomSearch, HyperModel
    from tensorflow.keras.layers import Input
    import sys
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    import numpy as np
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import RobustScaler
    import tensorflow as tf
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from sklearn.preprocessing import RobustScaler
    import joblib









ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_cook = pd.read_csv('cook_data_cleaned.csv')
df_hepworth = pd.read_csv('hepworth_data_cleaned.csv')



datasets =  [df_cook, df_hepworth]
for data in datasets:
    data['date'] = pd.to_datetime(df_hepworth['date'])
    data.set_index('date', inplace =True)
    data.interpolate(method='ffill', inplace=True)
    



# df_hepworth = df_hepworth[enhanced_features]
# df_cook = df_cook[enhanced_features]
    

scaler_path = 'robust_scaler.joblib'
scaler = joblib.load(scaler_path)


/home/harvey/.local/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


np.random.seed(43)
tf.random.set_seed(43)
scaler_path = 'robust_scaler.joblib'
scaler = joblib.load(scaler_path)
# Load pre-trained model
model_path = 'best_model_fold_4.keras'
model = tf.keras.models.load_model(model_path)

# Make only the last few layers trainable
for layer in model.layers[:-2]:
    layer.trainable = False
for layer in model.layers[-2:]:
    layer.trainable = True


features = df_cook.drop(['fostac', 'date'], axis=1)
targets = df_cook['fostac']
dates = df_cook['date']





# Determine indices for manual split
num_samples = len(features)
half_point = num_samples // 2

# Initialize variables to collect results
true_values = []
predicted_values = []
prediction_dates = []

# Setup Early Stopping and Reduce LR callbacks
early_stopping = EarlyStopping(monitor='mae', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='mae', factor=0.2, patience=5)
import numpy as np

# Loop over the dataset twice to switch training and validation sets
for i in range(2):  # Two iterations
    if i == 0:
        train_index = np.arange(0, half_point)
        val_index = np.arange(half_point, num_samples)
    else:
        train_index = np.arange(half_point, num_samples)
        val_index = np.arange(0, half_point)

    # Randomly select 5 indices from each half to add to the other half
    random_indices_from_train = np.random.choice(train_index, 10, replace=False)
    random_indices_from_val = np.random.choice(val_index, 10, replace=False)

    # Add these indices to the opposing set
    train_index = np.concatenate((train_index, random_indices_from_val))
    val_index = np.concatenate((val_index, random_indices_from_train))

    # Ensure validation set does not include the new train indices
    val_index = np.setdiff1d(val_index, random_indices_from_val)

    # Select the data for training and validation
    X_train, X_val = features.iloc[train_index], features.iloc[val_index]
    y_train, y_val = targets.iloc[train_index], targets.iloc[val_index]
    dates_train, dates_val = dates.iloc[train_index], dates.iloc[val_index]

    # Scaling
    scaler = joblib.load(scaler_path)
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Fit the model
    model.fit(X_train_scaled, y_train, epochs=130, batch_size=4,
              validation_data=(X_val_scaled, y_val), callbacks=[early_stopping, reduce_lr])

    # Predict only on validation sets
    val_predictions = model.predict(X_val_scaled).flatten()

    # Collect results for plotting
    true_values.extend(y_val.tolist())
    predicted_values.extend(val_predictions.tolist())
    prediction_dates.extend(dates_val.tolist())

# Continue with sorting and plotting


# Sort all by dates for consistent plotting
sorted_indices = np.argsort(prediction_dates)
sorted_dates = np.array(prediction_dates)[sorted_indices]
sorted_true_values = np.array(true_values)[sorted_indices]
sorted_predicted_values = np.array(predicted_values)[sorted_indices]

plt.figure(figsize=(15, 7))
plt.plot(sorted_dates, sorted_true_values, label='True Values', color='blue', marker='o', linestyle='-')
plt.plot(sorted_dates, sorted_predicted_values, label='Predicted Values', alpha=0.7, color='red', marker='o', linestyle='-')
plt.legend()
plt.title('Comparison of True and Predicted Values (Validation Only)')
plt.xlabel('Date')
plt.ylabel('Values')

# Show the plot
plt.show()

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 7], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_74'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']